# Function Calling

The most powerful feature of chat completion is the ability to call functions from the model. This allows you to create a chat bot that can interact with your existing code, making it possible to automate business processes, create code snippets, and more.
With Semantic Kernel, we simplify the process of using function calling by automatically describing your functions and their parameters to the model and then handling the back-and-forth communication between the model and your code.
When using function calling, however, it's good to understand what's actually happening behind the scenes so that you can optimize your code and make the most of this feature.

## How auto function calling works

When you call a function from the model, the model generates a JSON object that describes the function and its parameters. This JSON object is then passed to your code, which can then execute the function with the provided parameters.
The model is able to generate this JSON object because it has been trained on a large dataset of code and natural language, allowing it to understand the structure of functions and their parameters.
The model also has a built-in understanding of the types of parameters that functions can take, which allows it to generate the correct JSON object for the function being called.
The model uses a combination of natural language processing and code generation techniques to create the JSON object, which is then passed to your code for execution.

When you make a request to a model with function calling enabled, Semantic Kernel performs the following steps:

![FunctionCallingProcessSk](../../assets/images/functioncalling.png)

## Setting Up the Environment

First, let's import the necessary libraries and set up our environment. We'll use Semantic Kernel's Azure OpenAI connector to work with the OpenAI models that support function calling.

In [ ]:
# Import necessary libraries
import os
import json
import asyncio
from typing import Dict, Any

from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import (
    AzureChatCompletion,
    AzureChatPromptExecutionSettings
)

from semantic_kernel.functions import kernel_function, KernelArguments
from semantic_kernel.contents.chat_history import ChatHistory
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior

from dotenv import load_dotenv

# Load environment variables
load_dotenv()

chat_completion_service = AzureChatCompletion(
            service_id="azure_openai",
            deployment_name=os.environ.get("AZURE_OPENAI_COMPLETION_DEPLOYMENT_NAME"),
            api_key=os.environ.get("AZURE_OPENAI_API_KEY"),
            endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT"),
            api_version=os.environ.get("AZURE_OPENAI_API_VERSION")
        )

At a high-level, a plugin is a group of functions that can be exposed to AI apps and services. The functions within plugins can then be orchestrated by an AI application to accomplish user requests. Within Semantic Kernel, you can invoke these functions automatically with function calling.


## Creating a Plugin with Functions

Let's create a Weather plugin that includes two functions: one to get current weather data and another to get a weather forecast. In a real implementation, these functions would call external APIs, but for demonstration purposes, we'll return mock data.

In [ ]:
# Define a plugin with functions to get weather data and forecast
class WeatherPlugin:
    """
    A plugin to get weather data and forecast
    """
    # Define functions for function calling
    @kernel_function(name="get_weather_data", description="Get weather data for a specific location")
    async def get_weather_data(self, location: str) -> str:
        """
        Stub function to get weather data for a specific location
        In a real implementation, this would call a weather API
        """
        print(f"Getting weather data for {location}")
        weather_data = {
            "temperature": 25,  # Celsius
            "cloud_cover": 0.2,  # 20%
            "irradiance": 800,  # W/m²
            "precipitation_chance": 0.1,  # 10%
        }
        return json.dumps(weather_data)

    @kernel_function(name="get_weather_forecast", description="Get weather forecast for a specific location for the next day")
    async def get_weather_forecast(self, location: str) -> str:
        """
        Stub function to get weather forecast for a specific location
        In a real implementation, this would call a weather API
        """
        print(f"Getting weather forecast for {location}")
        forecast_data = {
            "temperature": 28,  # Celsius
            "cloud_cover": 0.1,  # 10%
            "irradiance": 900,  # W/m²
            "precipitation_chance": 0.05,  # 5%
        }
        return json.dumps(forecast_data)

## Setting Up the Kernel

Now, let's set up the Semantic Kernel and add our chat completion service and weather plugin. The kernel serves as the central orchestrator for our application.

In [ ]:
# Initialize the kernel
kernel = Kernel()

# Add the chat completion service created above to the kernel
kernel.add_service(chat_completion_service)

# Add the weather plugin to the kernel
# The plugin is a group of functions that can be exposed to AI apps and services
kernel.add_plugin(WeatherPlugin(), plugin_name="weather_plugin")

## Testing Function Calling

Now let's test our function calling capabilities. We'll create a chat history, add a user message asking about the weather, and then use our chat completion service with auto function calling enabled.

In [ ]:
# Initialize the chat history
chat_history = ChatHistory()

user_input = "What is the weather like in Seattle?"
system_input = "You are a helpful assistant that provides weather information. Keep it short and concise."
chat_history.add_user_message(user_input)

execution_settings = AzureChatPromptExecutionSettings()
execution_settings.function_choice_behavior = FunctionChoiceBehavior.Auto()

response = await chat_completion_service.get_chat_message_content(
    chat_history=chat_history,
    kernel=kernel,
    settings=execution_settings,
)

print(f"Assistant: {response}")

# Add the assistant's response to the chat history
chat_history.add_assistant_message(response.content)

chat_history.add_user_message("What is the weather like in Seattle tomorrow?")
response = await chat_completion_service.get_chat_message_content(
    chat_history=chat_history,
    kernel=kernel,
    settings=execution_settings,
)
print(f"Assistant: {response}")

## Creating More Complex Functions

Now let's create some additional functions that are more complex and demonstrate how they can be used together in a real-world scenario.

In [ ]:
class WeatherAnalysisPlugin:
    """
    A plugin to analyze weather data and provide recommendations
    """
    @kernel_function(name="analyze_weather_impact", description="Analyze how weather will impact outdoor activities")
    async def analyze_weather_impact(self, weather_data: str, activity_type: str) -> str:
        """
        Analyze weather data and provide recommendations for outdoor activities
        Args:
            weather_data: JSON string containing weather information
            activity_type: Type of activity (hiking, picnic, sports, etc.)
        """
        print(f"Analyzing weather impact for {activity_type}")
        # In a real implementation, this would analyze the weather data and return recommendations
        weather = json.loads(weather_data)
        
        analysis = {
            "activity": activity_type,
            "suitable": weather["temperature"] > 20 and weather["precipitation_chance"] < 0.3,
            "recommendations": [],
        }
        
        if weather["temperature"] > 30:
            analysis["recommendations"].append("Bring plenty of water due to high temperature")
        if weather["cloud_cover"] < 0.3:
            analysis["recommendations"].append("Bring sunscreen as UV exposure will be high")
        if weather["precipitation_chance"] > 0.1:
            analysis["recommendations"].append("Consider bringing rain gear just in case")
            
        return json.dumps(analysis)

# Add the new plugin to the kernel
kernel.add_plugin(WeatherAnalysisPlugin(), plugin_name="weather_analysis_plugin")

## Testing Complex Function Interactions

Let's test how the model can chain multiple function calls together to respond to a more complex question.

In [ ]:
# Create a new chat history for complex interaction
complex_chat = ChatHistory()

# Ask a more complex question that requires multiple function calls
complex_chat.add_user_message("I'm planning a hiking trip in Seattle tomorrow. Is the weather going to be good for hiking?")

# Use the same execution settings with auto function calling
complex_response = await chat_completion_service.get_chat_message_content(
    chat_history=complex_chat,
    kernel=kernel,
    settings=execution_settings,
)

print(f"Assistant: {complex_response}")

## Understanding Function Calling in the Background

Let's examine what's happening in the background during function calling. We'll set up a special execution setting that lets us see the actual function calls being made.

In [ ]:
# Create a chat history to observe function calls
observation_chat = ChatHistory()
observation_chat.add_user_message("What's the temperature in Paris right now, and will it be good for a picnic tomorrow?")

# Create execution settings with debug info
debug_settings = AzureChatPromptExecutionSettings()
debug_settings.function_choice_behavior = FunctionChoiceBehavior.Auto()
debug_settings.extension_data = {"include_raw_responses": True}  # This depends on the specific implementation

# Get response with debug information
debug_response = await chat_completion_service.get_chat_message_content(
    chat_history=observation_chat,
    kernel=kernel,
    settings=debug_settings,
)

# Print the debug information if available
if hasattr(debug_response, 'metadata') and debug_response.metadata:
    print("Function calling metadata:")
    print(debug_response.metadata)
    
print(f"\nFinal response: {debug_response.content}")

## Optional Exercise: Creating Your Own Plugin and Testing Function Calling

Now it's your turn! Create a new plugin that provides useful functions for a specific domain of your choice. Then test how the model uses these functions in response to user queries.

Here's a template to get you started:

In [ ]:
# Create your own plugin
class MyCustomPlugin:
    """
    Description of your custom plugin
    """
    @kernel_function(name="my_function1", description="Description of what this function does")
    async def my_function1(self, param1: str, param2: str = "default") -> str:
        """
        Detailed description of what this function does
        
        Args:
            param1: Description of param1
            param2: Description of param2
        """
        # Your function implementation here
        print(f"Function called with {param1} and {param2}")
        return json.dumps({"result": f"Processed {param1} with {param2}"})
    
    @kernel_function(name="my_function2", description="Description of what this function does")
    async def my_function2(self, input_data: str) -> str:
        """Another function implementation"""
        # Your function implementation here
        return f"Result: {input_data}"

# Add your plugin to the kernel
# kernel.add_plugin(MyCustomPlugin(), plugin_name="my_custom_plugin")

# Test your plugin with a user query
# my_chat = ChatHistory()
# my_chat.add_user_message("A query that would trigger your custom functions")
# 
# my_response = await chat_completion_service.get_chat_message_content(
#     chat_history=my_chat,
#     kernel=kernel,
#     settings=execution_settings,
# )
# 
# print(f"Response: {my_response.content}")

## Advanced Challenge: Multi-Function Conversation Agent

Now that you understand how function calling works, here's a more advanced challenge:

Create a full-featured conversational agent that can:

1. Use multiple functions from different plugins to answer complex queries
2. Maintain conversation context across multiple turns
3. Handle errors gracefully if a function call fails
4. Provide helpful responses that synthesize information from multiple function calls

Example implementation areas:
- A travel assistant that checks weather, flight status, and hotel availability
- A personal finance assistant that can check balances, analyze spending, and suggest budgets
- A health and wellness assistant that tracks activity, suggests workouts, and monitors nutrition

Bonus points if you implement:
- Function call tracing/logging
- User authentication for sensitive functions
- Fallback mechanisms when functions aren't available

Happy coding!